In [1]:
import os
from openai import OpenAI
import json


api_key = "sk-proj-"

client = OpenAI(api_key=api_key)

##### sample 1

In [2]:
def get_current_weather(location, unit="fahrenheit"):
    """Get the current weather in a given location"""
    
    # Assume the temperature in fahrenheit and convert if necessary
    temperature_fahrenheit = 72
    
    if location['unit'] == "celsius":
        temperature = (temperature_fahrenheit - 32) * 5.0/9.0
    else:
        temperature = temperature_fahrenheit

    weather_info = {
        "location":location['location'],
        "temperature": round(temperature, 2),
        "unit": location['unit'],  # Always set unit to "celsius" for Celsius temperature
        "forecast": ["sunny", "windy"],
    }
    return json.dumps(weather_info)

In [3]:
# define a function
functions = [
    {
        "name": "get_current_weather",
        "description": "Get the current weather in a given location",
        "parameters": {
            "type": "object",
            "properties": {
                "location": {
                    "type": "string",
                    "description": "The city and state, e.g. San Francisco, CA",
                },
                "unit": {
                    "type": "string",
                    "enum": ["celsius", "fahrenheit"]},
            },
            "required": ["location"],
        },
    }
]

#####  fuction call = auto  prompt = yes  response = yes

In [4]:
messages = [
    {
        "role": "user",
        "content": "What's the weather like in Boston?"
    }
]

In [5]:

response = client.chat.completions.create(
    model="gpt-3.5-turbo",  # gpt-3.5-turbo-0613 deprecated
    messages=messages,
    functions=functions,
    function_call="auto", 
    temperature=0
)

In [6]:
print(response)

ChatCompletion(id='chatcmpl-9igmgezULuDP06kptZlFCoGMO8VqX', choices=[Choice(finish_reason='function_call', index=0, message=ChatCompletionMessage(content=None, role='assistant', function_call=FunctionCall(arguments='{"location":"Boston","unit":"celsius"}', name='get_current_weather')), logprobs=None)], created=1720437922, model='gpt-3.5-turbo-0125', object='chat.completion', usage=CompletionUsage(completion_tokens=20, prompt_tokens=82, total_tokens=102), system_fingerprint=None)


In [7]:
response_message = response.choices[0].message
response_message

ChatCompletionMessage(content=None, role='assistant', function_call=FunctionCall(arguments='{"location":"Boston","unit":"celsius"}', name='get_current_weather'))

In [8]:
response_message.function_call

FunctionCall(arguments='{"location":"Boston","unit":"celsius"}', name='get_current_weather')

In [9]:
json.loads(response_message.function_call.arguments)

{'location': 'Boston', 'unit': 'celsius'}

In [10]:
args = json.loads(response_message.function_call.arguments)

In [11]:
get_current_weather(args)

'{"location": "Boston", "temperature": 22.22, "unit": "celsius", "forecast": ["sunny", "windy"]}'

#####  fuction call = function  prompt = yes  response = yes

In [17]:
messages = [
    {
        "role": "user",
        "content": "What's the weather like in Boston!",
    }
]
response = client.chat.completions.create(
    model="gpt-3.5-turbo",  # gpt-3.5-turbo-0613 deprecated
    messages=messages,
    functions=functions,
    function_call={"name": "get_current_weather"},
    temperature=0
)
print(response)

ChatCompletion(id='chatcmpl-9ih1XTqiAhcrvvqptq8VmyxPZEm7V', choices=[Choice(finish_reason='stop', index=0, message=ChatCompletionMessage(content=None, role='assistant', function_call=FunctionCall(arguments='{"location":"Boston","unit":"celsius"}', name='get_current_weather')), logprobs=None)], created=1720438843, model='gpt-3.5-turbo-0125', object='chat.completion', usage=CompletionUsage(completion_tokens=10, prompt_tokens=92, total_tokens=102), system_fingerprint=None)


##### Sample 2

#https://www.datacamp.com/tutorial/open-ai-function-calling-tutorial

In [41]:
custom_functions = [
    {
        'name': 'extract_student_info',
        'description': 'Get the student information from the body of the input text',
        'parameters': {
            'type': 'object',
            'properties': {
                'name': {
                    'type': 'string',
                    'description': 'Name of the person'
                },
                'major': {
                    'type': 'string',
                    'description': 'Major subject.'
                },
                'school': {
                    'type': 'string',
                    'description': 'The university name.'
                },
                'grades': {
                    'type': 'integer',
                    'description': 'GPA of the student.'
                },
                'club': {
                    'type': 'string',
                    'description': 'School club for extracurricular activities. '
                }
                
            }
        }
    },
    {
        'name': 'extract_school_info',
        'description': 'Get the school information from the body of the input text',
        'parameters': {
            'type': 'object',
            'properties': {
                'name': {
                    'type': 'string',
                    'description': 'Name of the school.'
                },
                'ranking': {
                    'type': 'integer',
                    'description': 'QS world ranking of the school.'
                },
                'country': {
                    'type': 'string',
                    'description': 'Country of the school.'
                },
                'no_of_students': {
                    'type': 'integer',
                    'description': 'Number of students enrolled in the school.'
                }
            }
        }
    }
]

In [42]:
school_1_description = "Stanford University is a private research university located in Stanford, California, United States. It was founded in 1885 by Leland Stanford and his wife, Jane Stanford, in memory of their only child, Leland Stanford Jr. The university is ranked #5 in the world by QS World University Rankings. It has over 17,000 students, including about 7,600 undergraduates and 9,500 graduates23. "
student_1_description = "David Nguyen is a sophomore majoring in computer science at Stanford University. He is Asian American and has a 3.8 GPA. David is known for his programming skills and is an active member of the university's Robotics Club. He hopes to pursue a career in artificial intelligence after graduating."


In [45]:
description = [student_1_description, school_1_description,a]
for i in description:
    response = client.chat.completions.create(
        model = 'gpt-3.5-turbo',
        messages = [{'role': 'user', 'content': i}],
        functions = custom_functions,
        function_call = 'auto'
    )

    # Loading the response as a JSON object
    json_response = json.loads(response.choices[0].message.function_call.arguments)
    print(json_response)

{'name': 'David Nguyen', 'major': 'Computer Science', 'school': 'Stanford University', 'grades': 3.8, 'club': 'Robotics Club'}
{'name': 'Stanford University', 'ranking': 5, 'country': 'United States', 'no_of_students': 17000}
{'name': 'David Nguyen', 'major': 'Computer Science', 'school': 'Stanford University', 'grades': 3.8, 'club': 'Robotics Club'}


#####  fuction call = auto  prompt = no  response = no

In [12]:
messages = [
    {
        "role": "user",
        "content": "hi!",
    }
]
response = client.chat.completions.create(
    model="gpt-3.5-turbo",  # gpt-3.5-turbo-0613 deprecated
    messages=messages,
    functions=functions,
    function_call="auto", 
    temperature=0
)
print(response)

ChatCompletion(id='chatcmpl-9igqIeR0Y7YWQiX9oIB0sbYgrgNjT', choices=[Choice(finish_reason='stop', index=0, message=ChatCompletionMessage(content='Hello! How can I assist you today?', role='assistant', function_call=None), logprobs=None)], created=1720438146, model='gpt-3.5-turbo-0125', object='chat.completion', usage=CompletionUsage(completion_tokens=10, prompt_tokens=76, total_tokens=86), system_fingerprint=None)


In [13]:
messages = [
    {
        "role": "user",
        "content": "hi!",
    }
]
response = client.chat.completions.create(
    model="gpt-3.5-turbo",  # gpt-3.5-turbo-0613 deprecated
    messages=messages,
    functions=functions,
    function_call="none", 
    temperature=0
)
print(response)

ChatCompletion(id='chatcmpl-9igr9bP0Ih2ssJvTJU8xajQrZPu4V', choices=[Choice(finish_reason='stop', index=0, message=ChatCompletionMessage(content='Hello! How can I assist you today?', role='assistant', function_call=None), logprobs=None)], created=1720438199, model='gpt-3.5-turbo-0125', object='chat.completion', usage=CompletionUsage(completion_tokens=9, prompt_tokens=77, total_tokens=86), system_fingerprint=None)


#####  fuction call = none prompt = ok  response = no

In [14]:
messages = [
    {
        "role": "user",
        "content": "What's the weather in Boston?",
    }
]
response = client.chat.completions.create(
    model="gpt-3.5-turbo",  # gpt-3.5-turbo-0613 deprecated
    messages=messages,
    functions=functions,
    function_call="none", 
    temperature=0
)
print(response)

ChatCompletion(id='chatcmpl-9igtcgWH7aUHhLK7FCeYqx1TLHnc7', choices=[Choice(finish_reason='stop', index=0, message=ChatCompletionMessage(content='Sure! Let me check the current weather in Boston for you.', role='assistant', function_call=None), logprobs=None)], created=1720438352, model='gpt-3.5-turbo-0125', object='chat.completion', usage=CompletionUsage(completion_tokens=13, prompt_tokens=82, total_tokens=95), system_fingerprint=None)


#####  fuction call = function  prompt = no  response = random

In [15]:
messages = [
    {
        "role": "user",
        "content": "hi!",
    }
]
response = client.chat.completions.create(
    model="gpt-3.5-turbo",  # gpt-3.5-turbo-0613 deprecated
    messages=messages,
    functions=functions,
    function_call={"name": "get_current_weather"},
    temperature=0
)
print(response)

ChatCompletion(id='chatcmpl-9igwJVaB70NaatQSBJzAoIl7Z6zRE', choices=[Choice(finish_reason='stop', index=0, message=ChatCompletionMessage(content=None, role='assistant', function_call=FunctionCall(arguments='{"location":"San Francisco, CA"}', name='get_current_weather')), logprobs=None)], created=1720438519, model='gpt-3.5-turbo-0125', object='chat.completion', usage=CompletionUsage(completion_tokens=8, prompt_tokens=86, total_tokens=94), system_fingerprint=None)
